# SpaCy

## Lavorare su grandi quantità di dati

Dopo i primi passi in questa parte verrà usata la libreria su grandi quantità di dati, mostrate le strutture e le potenzialità degli approcci statistico e basato sulle regole e la loro combianzione.

<br>

![struct](https://spacy.io/images/architecture.svg)

<br>

### Vocab

SpaCy conserva tutti i dati condivisi in un oggetto *Vocab*. Per dati non intendiamo solo le parole ma anche i tag, le entità, gli schemi e le etichette.

Il *Vocab* è una tabella di lookup, quando qualcosa viene caricato genera un ID con *codifica hash* così che possa essere usato sia per estrarre ID mediante la parola che la parola mediandte l'ID. Gli ID sono univoci, se qualcosa occorre più volte nei documenti non dovrà essere ricaricato.

> N.B. Gli hash ID non sono reversibili, se una stringa non sta nel Vocab non possiamo generarla/caricarla mediante un hash ID

In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
nlp.vocab.strings.add("coffee")

coffee_hash = nlp.vocab.strings["coffee"]
coffee_string = nlp.vocab.strings[coffee_hash]

print("coffee hash from string: ", coffee_hash)
print("coffee string from hash: ", coffee_string)

doc = nlp("I love coffee")
print("hash value:", doc.vocab.strings["coffee"])

coffee hash from string:  3197928453018144401
coffee string from hash:  coffee
hash value: 3197928453018144401


### Lexemes

I *Lexemes* sono voci del vocabolario indipendenti dal contesto, possono contenere informazioni lessicali ma non tag, entità, ... che dipendono dal contesto.

<br>

![lexeme](https://spacy.io/images/vocab_stringstore.svg)

<br>

Come si vede nell'immagine un documento contiente token e tutti i loro attributi, questi sono legati al vocabolario che contiene i lessemi, questa tabella di lookup è a sua volta collegata tramite hash ID allo string store.

> `is_alpha` nella cella successiva si riferisce a *is alphabetical*.

In [3]:
lexeme = nlp.vocab["coffee"]

print(lexeme.text, lexeme.orth, lexeme.is_alpha)

coffee 3197928453018144401 True


## Doc

Il `Doc` è la struttura centrale di SpaCy, viene creato in automatico quando si passa un testo nell'oggetto *nlp* ma può anche essere creato manualmente.

Nel prossimo esempio viene creato un `Doc` di tre parole, ogni *token* ha un attributo *spaces* ovvero un booleano che indica se la parola è seguita o no da uno spazio.

In [4]:
nlp = spacy.blank("en")

words = ["Hello", "world", "!"]
spaces = [True, False, False]

from spacy.tokens import Doc

doc = Doc(nlp.vocab, words, spaces)

Lo `Span` è una parte del `Doc`, consiste in uno o più token. Per creare uno `Span` bisogna passare come argomenti il `Doc` di riferimento e gli indici del token iniziale e finale che si vogliono inglobare nello span.

<br>

![span](https://course.spacy.io/span_indices.png)

<br>

> gli Span possono essere inseriti manualmente dentro le entità del documento

In [5]:
from spacy.tokens import Span

span = Span(doc, 0, 2)

print(span)

span_w_labels = Span(doc, 0, 2, label = "GREETING")

print(span_w_labels)

doc.ents = [span_w_labels]

print(doc.ents)

Hello world
Hello world
(Hello world,)


alcuni trucchetti per usare al meglio `Doc` e `Span`:

* se hai bisogno delle stringhe come output del tuo processo, converti il *doc* il più tardi possibile, potresti perdere delle relazioni interne all'oggetto
* usa gli attributi interni agli oggetti (come `token.i` per l'indice del token)
* non dimenticare di passarili nel vocabolario condiviso

## Word Vectors

SpaCy è in grado di confrontare documenti, span e token e dirci quanto sono simili. Infatti questi oggetti hanno un metodo `.similarity`. La similarità è espressa in un range [0,1], dove 1 è uguale (o molto simile).

Nel prossimo esempio calcoleremo la similarità tra frasi e la similarità tra parole (token.)

> N.B. per applicare la similarità bisogna usare pipeline di SpaCy medie o grandi, quele contrassegnate con il suffisso *_lg* (large) o *_md* (medium). Quella usata in precedenza aveva il suffisso *_sm* (small) e non consente di valutare la similarità.

In [6]:
import spacy
nlp = spacy.load('en_core_web_md')

doc1 = nlp("I like pasta")
doc2 = nlp("My car is red")
doc3 = nlp("I like pizza")

print(doc1.similarity(doc2), "\t", doc1.similarity(doc3))
print(doc2[1].similarity(doc3[2]), "\t", doc1[2].similarity(doc3[2]))

0.19793302526048495 	 0.974312099000299
0.1790039986371994 	 0.6850197911262512


Precedentemente sono stati confrontati oggetti dello stesso tipo, *doc* e *doc* o *token* e *token* ma possiamo anche confrontare oggetti di tipo diverso come *doc* e *span*, *doc* e *token*, ...

Per calcolare la similarità SpaCy utilizza i *word vectors* sono rappresentazioni multidimensionali delle parole tramite vettori, per crearle vengono utilizzati algoritmi come [Word2Vec](https://en.wikipedia.org/wiki/Word2vec), per avere un vettore che rappresenti un documento o uno span SpaCy fa la media dei vettori delle parole contenute, chiaramente questo metodo predilige testi brevi. La misura di similarità che si usa solitamente con i testi è la [distanza del coseno](https://it.wikipedia.org/wiki/Coseno_di_similitudine).

<br>

![cosine](https://miro.medium.com/v2/resize:fit:720/format:webp/1*GK56xmDIWtNQAD_jnBIt2g.png)

<br>

Il calcolo della similarità può essere molto utile in molte applicazioni (es. rimozione di duplicati), ma quando una frase è "simile"? Non esiste una risposta oggettiva perchè dipende dal contesto, ad esempio le frasi *"Amo i gatti"* e *"Odio i gatti"* possono essere simili perchè si parla di gatti ma completamente opposte allo stesso tempo.

Se vogliamo accedere al vettore di un token basta usare il metodo `.vector` (ad esempio il vettore della parola pizza). Come si può vedere i word vector di SpaCy (default) hanno lunghezza 300.

In [7]:
print(doc3[2].vector.shape)
doc3[2].vector

(300,)


array([-0.20348 , -2.376   , -4.6129  ,  2.5783  ,  0.70192 , -3.9578  ,
       -2.7163  ,  1.2299  , -1.1297  ,  3.6804  ,  4.0516  , -0.96515 ,
        4.6248  ,  4.0276  ,  0.85267 , -1.823   , -0.16034 ,  2.3589  ,
        0.88145 , -1.0626  ,  2.0095  ,  3.7962  , -0.55322 ,  2.0663  ,
       -1.8375  ,  2.0172  , -1.1873  ,  1.0524  , -4.7658  ,  0.84146 ,
        3.1651  , -0.16254 ,  2.4484  , -0.17522 ,  1.1742  , -1.81    ,
        0.3595  ,  1.3136  , -1.0159  ,  0.97661 , -0.98931 ,  1.7808  ,
        4.1094  , -0.49558 ,  6.1189  ,  1.5496  , -2.2132  , -3.1217  ,
        0.96873 ,  0.92924 , -1.6941  , -4.1219  ,  0.82571 , -3.871   ,
       -0.32475 , -2.6923  , -0.52908 ,  2.7431  ,  2.4325  ,  0.24599 ,
        5.3924  , -1.6597  ,  1.8607  , -2.1747  ,  5.0726  ,  2.3184  ,
       -6.8177  , -0.34929 ,  0.58031 , -0.24007 , -7.3874  ,  1.2737  ,
       -0.58188 ,  0.36543 ,  3.5527  , -0.68641 ,  1.7008  , -1.1614  ,
        2.077   , -3.0627  , -1.3665  ,  2.8223  , 

## Combinare modelli statistici e regole

Combinare i due approcci del NLP è un attrezzo potente, vedremo come fare con SpaCy.

L'**approccio statistico** è utile quando c'è bisogno di generalizzare e abbiamo esempi su cui applicare/addestrare i modelli, ad esempio la NER è un task che và risolto con l'approccio statistico.

L'**approccio basato su regole** è utile quando bisogna mappare un numero finito di istanze, ad esempio gli alimenti che non può mangiare un intollerante al lattosio in un libro di ricette.

### PhraseMatcher

Il PhraseMatcher è un'evoluzione del classico matcher, ci consente di fare anche ricerca per keywords. Molto veloce ci dà un accesso diretto ai token.

In [11]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("Golden Retriever")
matcher.add("DOG", [pattern])
doc = nlp("I have a Golden Retriever")

# Iterate over the matches
for match_id, start, end in matcher(doc):
    # Get the matched span
    span = doc[start:end]
    print("Matched span:", span.text)
     # Get the span's root token and root head token
    print("Root token:", span.root.text)
    print("Root head token:", span.root.head.text)
    # Get the previous token and its POS tag
    print("Previous token:", doc[start - 1].text, doc[start - 1].pos_)

Matched span: Golden Retriever
Root token: Retriever
Root head token: have
Previous token: a DET
